In [3]:
import pyfeats
from typing import *
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import os

In [ ]:
from utils import *

llf = LowLevelFeatureExtractor(function=pyfeats.glcm_features, 
                               params={'ignore_zeros': True}, 
                               features_set=['features_mean', 'features_range'])

root_folder = "C:\\Users\\trong\\Documents\\skin_data"

In [ ]:
# Define image transformations
transform = transforms.Compose([
    transforms.Resize((128, 128)),  # Resize images to 128x128
    transforms.Grayscale(num_output_channels=1),    # Convert to grayscale
    llf,
    # transforms.ToTensor(),  # Convert to tensor
])

In [ ]:
train_dataset = CSVImageMetadataDataset(csv_file='./data/vaynen_train.csv', root_dir=root_folder, transform=transform)
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)   

test_dataset = CSVImageMetadataDataset(csv_file='./data/vaynen_test.csv', root_dir=root_folder, transform=transform)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=4)

# Check one batch
images, metadata, labels = next(iter(train_dataloader))
print(images.shape)  # Example: torch.Size([32, 3, 128, 128])
print(metadata.shape)  # Example: torch.Size([32, 6])  -> 6 metadata features
print(labels.shape)  # Example: torch.Size([32])

In [ ]:
model = SimpleNeuralNetwork(inputs = llf.get_features_size())

# Check the model's architecture
print(model)

In [ ]:
train_model(model, train_dataloader, epochs=100, features_set=llf.function.__name__)

In [ ]:
evaluate_model(model, test_dataloader, features_set=llf.function.__name__)
